In [1]:
import torch
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

MODEL_PATH = "Qwen/Qwen3-VL-4B-Instruct"  # or local_dir from snapshot_download

model = Qwen3VLForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    dtype=torch.float16,
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(MODEL_PATH)


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.91G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [2]:
text = "The tallest mountain in the world is"

In [8]:
model = Qwen3VLForConditionalGeneration.from_pretrained(
    MODEL_PATH, dtype=torch.float16, device_map="auto"
).eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
print(model)

Qwen3VLForConditionalGeneration(
  (model): Qwen3VLModel(
    (visual): Qwen3VLVisionModel(
      (patch_embed): Qwen3VLVisionPatchEmbed(
        (proj): Conv3d(3, 1024, kernel_size=(2, 16, 16), stride=(2, 16, 16))
      )
      (pos_embed): Embedding(2304, 1024)
      (rotary_pos_emb): Qwen3VLVisionRotaryEmbedding()
      (blocks): ModuleList(
        (0-23): 24 x Qwen3VLVisionBlock(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Qwen3VLVisionAttention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (mlp): Qwen3VLVisionMLP(
            (linear_fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (linear_fc2): Linear(in_features=4096, out_features=1024, bias=True)
            (act_fn): GELUTanh()
          )
        )
      )
 

In [9]:
processor = AutoProcessor.from_pretrained(MODEL_PATH)
tokenizer = processor.tokenizer

In [10]:
enc = tokenizer(text, return_tensors="pt")
input_ids = enc["input_ids"].to(model.device)
attention_mask = enc.get("attention_mask", None)
if attention_mask is not None:
    attention_mask = attention_mask.to(model.device)
# 2) Forward pass -> logits for each position
with torch.inference_mode():
    out = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = out.logits  # [batch, seq_len, vocab]

In [13]:
# 3) Get logits for "next token" = last position
next_logits = logits[0, -1, :].float()          # float32 for stable softmax
probs = torch.softmax(next_logits, dim=-1)      # [vocab]

k=5
# 4) Top-k
top_probs, top_ids = torch.topk(probs, k)

print(f"Prompt: {text!r}\nTop-{k} next tokens:")
for p, tid in zip(top_probs.tolist(), top_ids.tolist()):
    tok = tokenizer.decode([tid])
    print(f"  id={tid:<6} p={p:.4f} token={tok!r}")

Prompt: 'The tallest mountain in the world is'
Top-5 next tokens:
  id=10423  p=0.7627 token=' Mount'
  id=35508  p=0.0380 token=' Mt'
  id=220    p=0.0210 token=' '
  id=1304   p=0.0151 token=' __'
  id=30743  p=0.0138 token=' ____'


In [16]:
tokenizer.decode([10423, 35508])

' Mount Mt'

In [6]:
import torch

def qwen3vl_chat(messages, max_new_tokens=128, **gen_kwargs):
    inputs = processor.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt",
    )
    inputs.pop("token_type_ids", None)  # sometimes present depending on template
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.inference_mode():
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, **gen_kwargs)

    trimmed = [o[len(i):] for i, o in zip(inputs["input_ids"], out)]
    return processor.batch_decode(trimmed, skip_special_tokens=True)[0]


In [7]:
messages = [
    {"role": "user", "content": [{"type": "text", "text": "Give me 3 ideas for a weekend in Berlin."}]}
]

print(qwen3vl_chat(messages, temperature=0.7, top_p=0.9))


Absolutely! Here are 3 unique and exciting weekend ideas for Berlin:

---

**1. Berlin’s Hidden Gems & Street Food Adventure**  
*Perfect for culture lovers & foodies*  
Start your weekend with a walk through lesser-known neighborhoods like Mitte’s backstreets, Neukölln’s artist quarters, or Kreuzberg’s bohemian alleys. Try street food at local markets like the **Berlin Flea Market (Berliner Markthalle)** or **Kiez-Markt**. Then, explore Berlin’s underground music scene with a late-night gig at a tiny club like **Kellerclub**


In [8]:
import torch
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

MODEL_ID = "Qwen/Qwen3-VL-4B-Instruct"
k = 10
text = "The tallest mountain in the world is"

model = Qwen3VLForConditionalGeneration.from_pretrained(
    MODEL_ID, dtype=torch.float16, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(MODEL_ID)
tokenizer = processor.tokenizer  # <-- use tokenizer directly

# 1) Tokenize text -> input_ids
enc = tokenizer(text, return_tensors="pt")
input_ids = enc["input_ids"].to(model.device)
attention_mask = enc.get("attention_mask", None)
if attention_mask is not None:
    attention_mask = attention_mask.to(model.device)

# 2) Forward pass -> logits for each position
with torch.inference_mode():
    out = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = out.logits  # [batch, seq_len, vocab]

# 3) Get logits for "next token" = last position
next_logits = logits[0, -1, :].float()          # float32 for stable softmax
probs = torch.softmax(next_logits, dim=-1)      # [vocab]

# 4) Top-k
top_probs, top_ids = torch.topk(probs, k)

print(f"Prompt: {text!r}\nTop-{k} next tokens:")
for p, tid in zip(top_probs.tolist(), top_ids.tolist()):
    tok = tokenizer.decode([tid])
    print(f"  id={tid:<6} p={p:.4f} token={tok!r}")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Prompt: 'The tallest mountain in the world is'
Top-10 next tokens:
  id=10423  p=0.7627 token=' Mount'
  id=35508  p=0.0380 token=' Mt'
  id=220    p=0.0210 token=' '
  id=1304   p=0.0151 token=' __'
  id=30743  p=0.0138 token=' ____'
  id=7407   p=0.0131 token=' located'
  id=32671  p=0.0127 token=' ______'
  id=537    p=0.0119 token=' not'
  id=304    p=0.0099 token=' in'
  id=1083   p=0.0077 token=' also'


In [17]:
tok = processor.tokenizer
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

In [18]:
texts = ["The tallest mountain in the world is", "The tallest mountain"]
enc = tok(texts, padding=True, return_tensors="pt").to(model.device)

In [19]:
with torch.inference_mode():
    logits = model(**enc).logits  # [B, T, V]

attn = enc["attention_mask"]
last_nonpad = attn.sum(dim=1) - 1

def topk_from(pos_ids, k=5):
    out = []
    for i, pos in enumerate(pos_ids):
        probs = torch.softmax(logits[i, pos].float(), dim=-1)
        p, ids = torch.topk(probs, k)
        out.append([(tok.decode([tid]), float(pp)) for tid, pp in zip(ids.tolist(), p.tolist())])
    return out

In [20]:
naive = topk_from([logits.size(1)-1]*len(texts), k=5)          # uses padded last column
correct = topk_from(last_nonpad.tolist(), k=5)                 # uses last real token

print("\nNAIVE (using -1):", naive[1])
print("\nCORRECT (using last_nonpad):", correct[1])


NAIVE (using -1): [(' tallest', 0.15411292016506195), (' world', 0.05849466845393181), (' ', 0.018406163901090622), (' highest', 0.01702291890978813), (' mountain', 0.014789768494665623)]

CORRECT (using last_nonpad): [(' in', 0.7184106111526489), (' on', 0.23690669238567352), (' peak', 0.014910136349499226), (' range', 0.009043455123901367), (' is', 0.007735277526080608)]


In [27]:
enc

{'input_ids': tensor([[   785,  81617,  16301,    304,    279,   1879,    374],
        [   785,  81617,  16301, 151643, 151643, 151643, 151643]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 0, 0, 0, 0]], device='cuda:0')}

In [21]:
from transformers import AutoProcessor
import torch

MODEL_ID = "Qwen/Qwen3-VL-4B-Instruct"
tok = AutoProcessor.from_pretrained(MODEL_ID).tokenizer
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "right"

texts = ["The tallest mountain in the world is", "The tallest mountain"]
enc = tok(texts, padding=True, return_tensors="pt")
ids, am = enc["input_ids"], enc["attention_mask"]

i = 1  # visualize the SHORTER example (index 1)
tokens = tok.convert_ids_to_tokens(ids[i].tolist())
T = ids.size(1)

# combined allow mask: can attend if (not padding) AND (causal: key<=query)
allow = torch.zeros((T, T), dtype=torch.int)
for q in range(T):
    for k in range(T):
        allow[q, k] = int((k <= q) and (am[i, k].item() == 1))

print("Tokens (columns/keys):")
print(" ".join([t.replace("Ġ","▁") for t in tokens]))
print("\nMask (rows=query, cols=key): █=allowed  .=blocked\n")
for q in range(T):
    row = "".join("█" if allow[q, k] else "." for k in range(T))
    print(f"{q:02d} {tokens[q]:>12}  {row}")


Tokens (columns/keys):
The ▁tallest ▁mountain <|endoftext|> <|endoftext|> <|endoftext|> <|endoftext|>

Mask (rows=query, cols=key): █=allowed  .=blocked

00          The  █......
01     Ġtallest  ██.....
02    Ġmountain  ███....
03 <|endoftext|>  ███....
04 <|endoftext|>  ███....
05 <|endoftext|>  ███....
06 <|endoftext|>  ███....


In [1]:
# COMPLETE, RUNNABLE SNIPPET:
# 1) Loads Qwen3-VL with an attention backend that RETURNS attention weights
# 2) Builds a padded batch so attention_mask matters
# 3) Prints (a) the attention_mask, (b) tokens, and (c) top attention weights
#    for one layer/head/query position, WITH vs WITHOUT attention_mask

import gc
import torch
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor

MODEL_ID = "Qwen/Qwen3-VL-4B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

# ---- Load tokenizer/processor ----
processor = AutoProcessor.from_pretrained(MODEL_ID)
tok = processor.tokenizer

# Make padding possible (use EOS as PAD if needed)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "right"

# ---- Load model (KEY: attn_implementation='eager' so attentions are returned) ----
# If you already had a model loaded in this kernel, free it first:
for name in ["model"]:
    if name in globals():
        del globals()[name]
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

model = Qwen3VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    dtype=torch.float16 if device == "cuda" else torch.float32,
    device_map="auto" if device == "cuda" else None,
    attn_implementation="eager",   # <-- IMPORTANT: enables output_attentions
).eval()

# ---- Create a padded batch ----
texts = ["The tallest mountain in the world is", "The tallest mountain"]
enc = tok(texts, padding=True, return_tensors="pt")
enc = {k: v.to(model.device) for k, v in enc.items()}

i = 1  # visualize the shorter example (index 1)
tokens = tok.convert_ids_to_tokens(enc["input_ids"][i].tolist())
attn_mask = enc["attention_mask"][i].tolist()
last_nonpad = int(sum(attn_mask) - 1)

print("\nTOKENS (shorter example):")
for idx, (m, t) in enumerate(zip(attn_mask, tokens)):
    print(f"{idx:02d} mask={m} token={t!r}")

print(f"\nlast_nonpad index = {last_nonpad} (token={tokens[last_nonpad]!r})")

# ---- Function to print top attention weights for one head ----
def show_attention(with_mask: bool, layer=0, head=0, qpos=None, topk=10):
    if qpos is None:
        qpos = last_nonpad

    am = enc["attention_mask"] if with_mask else None

    with torch.inference_mode():
        out = model(
            input_ids=enc["input_ids"],
            attention_mask=am,
            output_attentions=True,
            return_dict=True,
        )

    if out.attentions is None:
        raise RuntimeError(
            "attentions=None. This usually means the attention backend can't return weights.\n"
            "Try ensuring attn_implementation='eager' at load time, and that you're on a recent transformers."
        )

    # out.attentions[layer]: [batch, heads, q_len, k_len]
    A = out.attentions[layer][i, head, qpos].float().detach().cpu()  # [k_len]
    vals, idx = torch.topk(A, k=min(topk, A.numel()))

    print(f"\n--- {'WITH' if with_mask else 'NO'} attention_mask | layer={layer} head={head} | query_pos={qpos} ({tokens[qpos]!r}) ---")
    for v, j in zip(vals.tolist(), idx.tolist()):
        print(f"attn={v:8.5f}  key_pos={j:02d}  key_tok={tokens[j]!r}  key_mask={attn_mask[j]}")

# ---- Compare WITHOUT vs WITH attention_mask ----
show_attention(with_mask=False, layer=0, head=0, qpos=last_nonpad, topk=12)
show_attention(with_mask=True,  layer=0, head=0, qpos=last_nonpad, topk=12)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


TOKENS (shorter example):
00 mask=1 token='The'
01 mask=1 token='Ġtallest'
02 mask=1 token='Ġmountain'
03 mask=0 token='<|endoftext|>'
04 mask=0 token='<|endoftext|>'
05 mask=0 token='<|endoftext|>'
06 mask=0 token='<|endoftext|>'

last_nonpad index = 2 (token='Ġmountain')

--- NO attention_mask | layer=0 head=0 | query_pos=2 ('Ġmountain') ---
attn= 0.75488  key_pos=00  key_tok='The'  key_mask=1
attn= 0.12561  key_pos=02  key_tok='Ġmountain'  key_mask=1
attn= 0.11945  key_pos=01  key_tok='Ġtallest'  key_mask=1
attn= 0.00000  key_pos=06  key_tok='<|endoftext|>'  key_mask=0
attn= 0.00000  key_pos=05  key_tok='<|endoftext|>'  key_mask=0
attn= 0.00000  key_pos=04  key_tok='<|endoftext|>'  key_mask=0
attn= 0.00000  key_pos=03  key_tok='<|endoftext|>'  key_mask=0

--- WITH attention_mask | layer=0 head=0 | query_pos=2 ('Ġmountain') ---
attn= 0.75488  key_pos=00  key_tok='The'  key_mask=1
attn= 0.12561  key_pos=02  key_tok='Ġmountain'  key_mask=1
attn= 0.11945  key_pos=01  key_tok='Ġtallest'

In [2]:
print(tok.decode([tok.convert_tokens_to_ids("Ġmountain")]))
# should print: " mountain"


 mountain


In [25]:
import gc
import torch

# delete your big GPU objects
del model
del processor
del enc
del logits

gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()


NameError: name 'model' is not defined